In [50]:
import numpy as np 
import pandas as pd
import statsmodels.api as sm
from scipy.special import gamma

In [51]:
cz = pd.read_csv('../data/raw/archive/cz_county.csv')
cz = cz.dropna()
cz['LMA/CZ'] = cz['LMA/CZ'].astype(str).str[:-2].astype(int)
idx = cz.groupby(['LMA/CZ'])['Labor Force'].idxmax()
cz = cz.loc[idx].reset_index(drop=True)
cz = cz[['LMA/CZ', 'County Name']].set_index('LMA/CZ')

In [52]:
df_wages = pd.read_csv('../data/processed/city_occ_wage/city_occ_w_2019.csv')
df_wages = df_wages.drop(columns=['other'])
df_wages['COMZONE'] = df_wages['COMZONE'].map(cz['County Name'])
df_wages = df_wages.set_index('COMZONE')

In [53]:
theta = 3
rho = 0.5
alpha = 7
eta = 1.65

w_ck = df_wages.fillna(0).to_numpy()

lambda_k = (w_ck ** theta).sum(axis=0)

T_ck_top = w_ck
T_ck_bottom = (
    gamma((theta - 1) / theta) * 
    (w_ck ** theta) * 
    (lambda_k ** (- rho)) *  
    ((lambda_k ** (1 - rho)).sum(axis=0) ** ((1 - theta) / theta))
)
T_ck = pd.DataFrame(np.log(T_ck_top / T_ck_bottom))
T_ck.columns = df_wages.columns
T_ck['COMZONE'] = df_wages.index
T_ck = pd.melt(T_ck, id_vars=['COMZONE'], var_name='occupation', value_name='wage')

/tmp/ipykernel_1916049/1365873024.py:17: RuntimeWarning: invalid value encountered in divide
  T_ck = pd.DataFrame(np.log(T_ck_top / T_ck_bottom))


In [55]:
model = sm.OLS.from_formula('wage ~ C(occupation) + C(COMZONE)', data=T_ck)
model = model.fit()
coefficients =  - model.params
T_ck['residual'] = model.resid

T_k = pd.DataFrame(coefficients[coefficients.index.str.contains('occupation')]).reset_index()
T_k['occupation'] = T_k['index'].str.extract(r'C\(occupation\)\[T\.(.*?)\]')
T_k = T_k.drop(columns=['index']).rename(columns={0: 'coefficient'})
T_k = T_k[['occupation', 'coefficient']]
T_k['coefficient'] = np.exp(T_k['coefficient'])

T_c = pd.DataFrame(coefficients[coefficients.index.str.contains('COMZONE')]).reset_index()
T_c['COMZONE'] = T_c['index'].str.extract(r'C\(COMZONE\)\[T\.(.*?)\]')
T_c = T_c.drop(columns=['index']).rename(columns={0: 'coefficient'})
T_c = T_c[['COMZONE', 'coefficient']]
T_c['coefficient'] = np.exp(T_c['coefficient'])

t_ck = T_ck.pivot(index='COMZONE', columns='occupation', values='residual')
t_ck = t_ck.apply(lambda x: np.exp(x))

T_c.to_csv('params/T_c.csv', index=False)
T_k.to_csv('params/T_k.csv', index=False)
t_ck.to_csv('params/t_ck.csv')